In [ ]:
import Data.List (intercalate)

data TableHeader = RowHeader | ColHeader | NoHeader deriving (Eq)
data TableCellEnvironment = DefaultCells | MathCells deriving (Eq)

renderTable :: TableHeader -> TableCellEnvironment -> [String] -> [[String]] -> String
renderTable h env cols rows =
    "\\begin{tabular}{|*{" ++ show (length cols) ++ "}{c|}}\n\\hline\n" ++
    joinRows (fmtCols (headerCell <$> cols) : (fmtCols <$> rows) ++ ["\\end{tabular}"])
    where
        headerCell c
            | h == RowHeader = boldCell c
            | otherwise      = c
        fmtCols (c : cs)
            | h == ColHeader = intercalate " & " (boldCell c : (cell <$> cs))
            | otherwise      = intercalate " & " (cell <$> (c : cs))
        joinRows = intercalate "\\\\\\hline\n"
        cell c
            | env == MathCells = "$" ++ c ++ "$"
            | otherwise        = c
        boldCell c = "\\textbf{" ++ cell c ++ "}"

In [ ]:
binaryWidth :: [Int] -> Int
binaryWidth = (+ 1) . floor . logBase 2.0 . fromIntegral . maximum

encodeBinary :: Int -> Int -> [Int]
encodeBinary width = padBinary width . reverse . revBinary
    where
        revBinary 0 = [0]
        revBinary n = (n `mod` 2) : revBinary (n `div` 2)
        padBinary width bs = replicate (width - length bs) 0 ++ bs

encodeBinaryAll :: [Int] -> [[Int]]
encodeBinaryAll ns = encodeBinary (binaryWidth ns) <$> ns

In [ ]:
prependHeaders :: [a] -> [[a]] -> [[a]]
prependHeaders (h:hs) (cs:css) = (h:cs) : prependHeaders hs css
prependHeaders _ _ = []

In [ ]:
fromZero = ((subtract 1 <$>) <$>)

transitions = fromZero [[2, 1, 5, 6, 2, 3], [4, 3, 1, 4, 6, 5], [6, 5, 3, 2, 4, 1]]
outputs = fromZero [[1, 2, 3, 3, 2, 1], [2, 3, 1, 2, 1, 3], [3, 1, 2, 1, 3, 2]]

printEncoded ts = (printCell <$>) <$> ts
    where
        printCell = (show =<<) . encode
        encode = encodeBinary (binaryWidth $ head ts)

In [ ]:
statesEncodedHeader = "$x_1x_2/Q_1Q_2Q_3$" : ((show =<<) <$> encodeBinaryAll [0..5])

-- Transitions encoded
putStrLn $ renderTable NoHeader DefaultCells statesEncodedHeader $
    prependHeaders ((show =<<) . encodeBinary 2 <$> [0..]) (printEncoded transitions)
    
-- Outputs encoded
putStrLn $ renderTable NoHeader DefaultCells statesEncodedHeader $
    prependHeaders ((show =<<) . encodeBinary 2 <$> [0..]) (printEncoded outputs) ++
    ["" : replicate 6 "$y_1y_2$"]

In [ ]:
-- State transitions
putStrLn $ renderTable NoHeader MathCells ["\\delta", "a_1", "a_2", "a_3", "a_4", "a_5", "a_6"]
    ((\(i, s) -> ("z_" ++ show i) : s) <$> zip [1..] (((("a_" ++) . show . (+ 1)) <$>) <$> transitions))
  
-- Outputs
putStrLn $ renderTable NoHeader MathCells ["\\lambda", "a_1", "a_2", "a_3", "a_4", "a_5", "a_6"]
    [ ["z_1", "w_1" , "w_2" , "w_3" , "w_3" , "w_2" , "w_1"]
    , ["z_2", "w_2" , "w_3" , "w_1" , "w_2" , "w_1" , "w_3"]
    , ["z_3", "w_3" , "w_1" , "w_2" , "w_1" , "w_3" , "w_2"] ]

-- Input encoding
putStrLn $ renderTable NoHeader MathCells ["", "x_1", "x_2"]
    [ ["z_1", "0", "0"]
    , ["z_2", "0", "1"]
    , ["z_3", "1", "0"] ]

-- Output encoding
putStrLn $ renderTable NoHeader MathCells ["", "y_1", "y_2"]
    [ ["w_1", "0", "0"]
    , ["w_2", "0", "1"]
    , ["w_3", "1", "0"] ]

-- State encoding
putStrLn $ renderTable NoHeader MathCells ["", "Q_1", "Q_2", "Q_3"]
    [ ["a_1", "0", "0", "0"]
    , ["a_2", "0", "0", "1"]
    , ["a_3", "0", "1", "0"]
    , ["a_4", "0", "1", "1"]
    , ["a_5", "1", "0", "0"]
    , ["a_6", "1", "0", "1"] ]